In [ ]:
import pandas as pd

In [ ]:
# Load airport-level daily data (weather + flights)
mart_ap_d_df = pd.read_csv('./csv_files/mart_airport_daily_milton.csv')
mart_ap_d_df.info()  # inspect schema and data types

In [ ]:
# Convert date column to datetime type for time-based operations
mart_ap_d_df['date'] = pd.to_datetime(mart_ap_d_df['date'])
mart_ap_d_df.info()  # confirm dtype change

In [ ]:
mart_ap_d_df.head()  # preview data

In [ ]:
# Calculate the average number of flights per day per airport during the pre-storm baseline window (Oct 1–7)

In [ ]:
# Define the pre-storm baseline window
baseline_start = "2024-10-01"
baseline_end = "2024-10-07"

# Filter data to baseline period and calculate average daily flights per airport
baseline_df = (
    mart_ap_d_df[mart_ap_d_df['date'].between(baseline_start, baseline_end)]
    .groupby('airport_code')['num_flights']
    .mean()
    .reset_index()
)

# Round and rename the column for clarity
baseline_df['num_flights'] = baseline_df['num_flights'].round()
baseline_df = baseline_df.rename(columns={'num_flights': 'baseline_flights'})

In [ ]:
baseline_df

In [ ]:
# Merge baseline values into full dataset to enable comparisons by date
df = mart_ap_d_df.merge(baseline_df, on='airport_code')

In [ ]:
# Identify recovery milestones for each airport: 
# Reopened: when operations resumed at 50% of baseline and <50% cancellations
# Fully Recovered: when operations reached 90% of baseline and <5% cancellations

In [ ]:
# Stage 1: Reopened — airport resumes meaningful operations
reopened_rows = df[
    (df['date'] >= "2024-10-10") &
    (df['pct_cancelled'] < 0.5) &  # fewer than half of flights canceled
    (df['num_flights'] >= 0.5 * df['baseline_flights'])  # at least 50% of baseline volume
]

first_reopened = (
    reopened_rows
    .groupby('airport_code')['date']
    .min()
    .reset_index()
    .rename(columns={'date': 'reopen_date'})
)

# Stage 2: Fully Recovered — near-normal operations resume
recovered_rows = df[
    (df['date'] >= "2024-10-10") &
    (df['pct_cancelled'] < 0.05) &  # nearly all scheduled flights operating
    (df['num_flights'] >= 0.9 * df['baseline_flights'])  # 90% of baseline volume
]

first_recovered = (
    recovered_rows
    .groupby('airport_code')['date']
    .min()
    .reset_index()
    .rename(columns={'date': 'recovery_date'})
)

In [ ]:
first_reopened

In [ ]:
first_recovered

In [ ]:
# Merge reopening and recovery dates into a single summary
status_df = pd.merge(first_reopened, first_recovered, on='airport_code', how='left')
status_df

In [ ]:
# Now I will build resilience metrics table: 
# 1. Days to recovery
# 2. Total % of flights cancelled for Oct 2024
# 3. Highest hourly avg windspeed

In [ ]:
# First I will calculate the days to reopen & days to recovery

In [ ]:
# Define the storm impact start date
storm_date = pd.to_datetime("2024-10-09")

# Calculate both metrics
status_df['days_to_reopen'] = (status_df['reopen_date'] - storm_date).dt.days
status_df['days_to_recovery'] = (status_df['recovery_date'] - storm_date).dt.days

# Preview to confirm
status_df

In [ ]:
# Second I will calculate the % of flights cancelled in Oct 2024

In [ ]:
monthly_stats = ( 
    df.groupby('airport_code')
    .agg({
        'num_flights': 'sum',
        'num_cancelled': 'sum'
    })
    .reset_index()
)

# Calculate percent of flights cancelled
monthly_stats['pct_cancelled_oct'] = (
    monthly_stats['num_cancelled'] / monthly_stats['num_flights']
).round(3)

monthly_stats

In [ ]:
# Finally I will calculate the peak wind speeds by airport

In [ ]:
# Peak wind speed = highest hourly average wind speed recorded per airport
peak_wind = (
    df.groupby('airport_code')['peak_wind_speed_kmh']
    .max()
    .reset_index()
)

peak_wind

In [ ]:
# Combine recovery data, cancellation rate, and peak wind speed into a single dataset
resilience_df = (
    status_df
    .merge(monthly_stats[['airport_code', 'pct_cancelled_oct']], on='airport_code')
    .merge(peak_wind, on='airport_code')
)

In [ ]:
resilience_df

In [ ]:
# Resilience Score Calculation

# The goal of this formula is to evaluate how effectively each airport recovered from Hurricane Milton.
# After exploring multiple options, I needed a score that fairly balanced:
#   - The intensity of the storm each airport experienced
#   - The speed and efficiency of operational recovery

# Original attempts used raw wind speed in the denominator,
# but this unintentionally favored airports with extreme wind conditions—even if they recovered poorly.

# To correct this, I applied the square root to peak wind speed.
# This still rewards resilience in stronger storms, but dampens the disproportionate benefit from extreme values.

# The numerator combines:
#   - days_to_recovery: how many days the airport took to return to baseline
#   - pct_cancelled_oct: the overall disruption in October, scaled by a factor of 10 to match recovery days on a similar scale

In [ ]:
# Final formula: lower scores indicate better resilience (fast recovery and minimal cancellations)
resilience_df['resilience_score'] = (
    (resilience_df['days_to_recovery'] + 
     (resilience_df['pct_cancelled_oct'] * 10)) /
    (resilience_df['peak_wind_speed_kmh'] ** 0.5 / 10)
).round(2)
# Rank by resilience (lower score is more resilient)
resilience_df = resilience_df.sort_values('resilience_score', ascending=True)

In [ ]:
resilience_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

In [ ]:
# Quick test plot to visualize % of flight cancellations around hurricane landfall
test_df = df[
    (df['date'] >= '2024-10-05') &
    (df['date'] <= '2024-10-17')
].copy()

# Pivot to wide format for line plotting
pivot_df = test_df.pivot(index='date', columns='airport_code', values='pct_cancelled')

pivot_df.plot(
    figsize=(12, 6),
    marker='o',
    title="% of Flights Cancelled by Airport (Oct 5–17)",
    ylabel="% Cancelled",
    xlabel="Date",
    grid=True
)
plt.tight_layout()

In [ ]:
# Create upgraded, more detailed version in seaborn

plot_df = df[                                       # Filter for Oct 5–17
    (df['date'] >= '2024-10-05') &                  
    (df['date'] <= '2024-10-17')
].copy()

# Highlight SRQ vs. other airports to show relative cancellation behavior
plot_df['highlight'] = plot_df['airport_code'].apply(
    lambda x: 'SRQ' if x == 'SRQ' else 'DAB, TPA, MCO, PIE'
)

sns.set_theme(style="whitegrid")                    # Set Seaborn theme
fig, ax = plt.subplots(figsize=(12, 6))             # Create figure and axis
ax.set_facecolor('#f0f0f0')                         # Light grey background inside plot

sns.lineplot(                                       # Line plot showing cancellation rate trends
    data=plot_df,
    x="date",
    y="pct_cancelled",
    hue="highlight",
    palette={
        "SRQ": "#FF6F00",                           # Highlight color for SRQ
        "DAB, TPA, MCO, PIE": "#4C72B0"
    },
    linewidth=2,
    marker="o",
    ax=ax
)

# Add hurricane landfall marker
landfall = pd.to_datetime("2024-10-09 20:30")
ax.axvline(landfall, color='darkred', linestyle='--', linewidth=1.5)
ax.text(landfall, 0.1, 'Landfall      \nOct 09, 8:30pm', ha='right', va='bottom', color='darkred', weight='bold')

# Titles and labels
ax.set_title("Flight Cancellations Surrounding Hurricane Milton", fontsize=14)
ax.set_xlabel("Date (2024)")
ax.set_ylabel("Cancellation Rate (%)")

# Y-axis formatting (percent style)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y * 100:.0f}%'))
ax.set_ylim(0, 1.05)

# X-axis date formatting
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

# Legend and layout
ax.legend(title="Airport Group", loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Daily flight cancellations from Oct 5–17 across five Florida airports.
# Hurricane Milton made landfall on Oct 9 (dashed line).
# While some airports recovered within days, SRQ remained heavily impacted long after the storm passed.

In [ ]:
# Bar chart: Resilience score by airport after Hurricane Milton

# Sort airports by resilience score (lower = better)
res_plot = resilience_df.sort_values('resilience_score', ascending=True).copy()

# Set up colors: highlight SRQ
bar_colors = ['#FF6F00' if code == 'SRQ' else '#4C72B0' for code in res_plot['airport_code']]

fig, ax = plt.subplots(figsize=(10, 6))             # Create figure and axes
ax.set_facecolor('#f0f0f0')                         # Light grey background inside chart

bars = ax.barh(                                     # Draw horizontal bars
    y=res_plot['airport_code'],
    width=res_plot['resilience_score'],
    color=bar_colors
)

# Annotate scores next to each bar
for i, (score, airport) in enumerate(zip(res_plot['resilience_score'], res_plot['airport_code'])):
    ax.text(score + 0.1, i, f"{score:.1f}", va='center', weight='bold')

# Title and axis labels
ax.set_title('Airport Resilience Scores After Hurricane Milton (October 2024)', fontsize=14)
ax.set_xlabel('Resilience Score (Lower = More Resilient)', fontsize=12)
ax.set_ylabel('Airport Code', fontsize=12)
ax.invert_yaxis()                                   # Airports with best scores appear at the top

plt.tight_layout()
plt.show()

In [ ]:
# Each bar represents a resilience score for airports affected by Hurricane Milton (October 2024).
# The score combines time to recovery and flight cancellation rate, normalized by peak wind speed (maximum hourly average wind speed).
# SRQ, which faced the strongest impact and infrastructure damage, remained closed significantly longer than its peers — resulting in the highest (worst) resilience score.

In [ ]:
# Prepare a polished summary table for export & README inclusion
recovery_table = resilience_df.copy()

# Rename columns for readability
recovery_table = recovery_table.rename(columns={
    'airport_code': 'Airport',
    'reopen_date': 'Date Reopened',
    'recovery_date': 'Date Recovered',
    'days_to_reopen': 'Days to Reopen',
    'days_to_recovery': 'Days to Recovery',
    'pct_cancelled_oct': '% Flights Cancelled (Oct)',
    'peak_wind_speed_kmh': 'Peak Wind (km/h)',
    'resilience_score': 'Resilience Score'
})

# Format numeric columns
recovery_table['% Flights Cancelled (Oct)'] = (recovery_table['% Flights Cancelled (Oct)'] * 100).round(1)
recovery_table['Resilience Score'] = recovery_table['Resilience Score'].round(2)
recovery_table['Peak Wind (km/h)'] = recovery_table['Peak Wind (km/h)'].round(1)

# Sort for presentation
recovery_table = recovery_table.sort_values('Resilience Score').reset_index(drop=True)

recovery_table

In [ ]:
# Export the cleaned-up dataframe as .csv
# recovery_table.to_csv("csv_files/airport_resilience_summary.csv", index=False)